In [1]:
import evaluate
from openai import AzureOpenAI
import os
import pandas as pd
import datasets
from sklearn.metrics import f1_score, precision_score, recall_score, classification_report
import evaluate

def compute_metrics(predictions, references, labels=None, pos_label=1, average="weighted", sample_weight=None, zero_division='warn'):
        f1 = f1_score(
            references, predictions, labels=labels, pos_label=pos_label, average=average, sample_weight=sample_weight
        )
        p = precision_score(
            references, predictions, labels=labels, pos_label=pos_label, average=average, sample_weight=sample_weight,
            zero_division=zero_division
        )
        r = recall_score(
            references, predictions, labels=labels, pos_label=pos_label, average=average, sample_weight=sample_weight,
            zero_division=zero_division
        )
        c = classification_report(
            references, predictions, labels=labels
        )
        print(c)
        return {"f1": float(f1) if f1.size == 1 else f1,
                "precision": float(p) if p.size == 1 else p,
                "recall": float(r) if r.size == 1 else r}

client = AzureOpenAI(azure_endpoint='https://oai-cbipm-01.openai.azure.com/',
                     api_key="cb0e51bd1e4a46fbb4043b8bcdd04cd7",
                     api_version="2023-12-01-preview")  # 2023-12-01-preview <- highest version number as of 1/12/23

deployment = "Deployment"

In [12]:
dataset = pd.read_csv("../psych_nlp/sentiment-analysis/data/sentences_MD-labels_GPT.csv")
# replace = {"neutral": 0, "negative": 1, "positive": 2}
# dataset["labels"] = dataset["MD_label"].map(replace)
# labels = dataset["labels"]
sentences = dataset["language"].to_json()

print(len(sentences))
print(sentences)

5398
{"0":"Patient very adamantly against hospitalization; states she is not suicidal and needs to go home to care for her dog","1":"Pt admits to sometimes thinking about death, wondering if she would be better off, but adamantly denies any thoughts of taking her own life, very much wants to live","2":"However this morning he is adamant that he wants to go to XXX, does not want to go to program and does not want to go home","3":"Pt continues to report adherence with his medication; cannot offer explanation as to why his symptoms were worse yesterday","4":"Adherent with medications, and lithium level of 0.7 indicates likely adherence as outpatient also.","5":"States he is adherent w\/ his outpatient XXX and HIV meds.","6":"Pt has been agitated throught her time in the ED, was given lorazepam 2mg po approximately 30 minutes ago, continues to yell and scream and just threw chairs in the ED, but is now resting","7":"She has been awake now for several hours and has become active and hyperve

In [19]:
messages = [{"role": "system", "content": "You are a medical doctor."},
            {"role": "user", "content": "As a medical doctor, you write many clinical notes about patients.\n"
                                        "Your task is to analyze the sentiment of a series of sentences you wrote about patients.\n"
                                        "For each sentence, what is your attitude towards the patient you wrote about?\n"
                                        "Please assign a sentiment score of negative, neutral, or positive for each sentence.\n"
                                        "Below are three example sentences in JSON format:\n"
                                        "{\"0\":\"55 yo male with XXX, h/o asthma, BIB police for threatening behavior, disorganization and paranoia in the setting of medication non-adherence\","
                                        "\"1\":\"Sister saw pt often while he was at XXX b/c she works the night shift and would visit him on her breaks; states he was always quite sweet, not agitated, but sometimes thought she was their  mother, or another sister\","
                                        "\"2\":\"She does not come regularly for her appointments and has poor compliance with tx care\"}"
                                        "Please provide your answer in JSON format."
             },
            {"role": "assistant", "content": "{\"0\":\"neutral\",\"1\":\"positive,\"2\":\"negative\"}"},
            {"role": "user", "content": f"Complete the same task with each of these sentences:\n{sentences}"}]

response = client.chat.completions.create(model=deployment, messages=messages, temperature=0, seed=42)

output = response.choices[0].message.content
# predictions = pd.read_json(output, orient="index")
# predictions.columns.values[0] = "predictions"
# predictions["predictions"] = predictions[].map(replace)
print(output)

{"0":"neutral","1":"neutral","2":"negative","3":"neutral","4":"positive","5":"positive","6":"negative","7":"neutral","8":"negative","9":"negative","10":"neutral","11":"positive","12":"negative","13":"negative","14":"neutral","15":"negative","16":"negative","17":"negative","18":"positive","19":"negative","20":"negative","21":"positive","22":"neutral","23":"neutral","24":"negative","25":"negative","26":"negative","27":"negative","28":"negative","29":"negative","30":"negative","31":"negative","32":"negative","33":"positive","34":"positive","35":"positive","36":"negative","37":"neutral","38":"negative"}


In [20]:
predictions = pd.read_json(output, orient="index")
predictions

/var/folders/xm/m4dy5bvs1fn7fb9wxjhsg4dc0000gn/T/ipykernel_72042/1616948092.py:1: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  predictions = pd.read_json(output, orient="index")


,0
0,neutral
1,neutral
2,negative
3,neutral
4,positive
5,positive
6,negative
7,neutral
8,negative
9,negative


In [21]:
compute_metrics(predictions[0], dataset["MD_label"])

              precision    recall  f1-score   support

    negative       0.59      0.87      0.70        15
     neutral       0.67      0.35      0.46        17
    positive       0.75      0.86      0.80         7

    accuracy                           0.64        39
   macro avg       0.67      0.69      0.65        39
weighted avg       0.65      0.64      0.62        39



{'f1': 0.6150434458126767,
 'precision': 0.6524864024864024,
 'recall': 0.6410256410256411}